<a href="https://colab.research.google.com/github/xbrucelee88x/2020/blob/master/20200722_alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install alpaca-trade-api

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 204kB 27.3MB/s 
     |████████████████████████████████| 1.2MB 37.5MB/s 
     |████████████████████████████████| 153kB 47.4MB/s 
     |████████████████████████████████| 256kB 38.7MB/s 
  Created wheel for alpha-vantage: filename=alpha_vantage-2.2.0-cp36-none-any.whl size=25705 sha256=aba3f45a3b3ae4d3695e5e99f8c3640e84577d513893c542dc8251e51c08cb1a
  Stored in directory: /root/.cache/pip/wheels/3a/fa/05/80b0afcbd2a4de8bd2a0c5cd77084e3d3a9cc12c48be9b7f84
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=63373638c3965439d69ee1454638063b45a13a69c124b28139dabfdcea993432
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built alpha-vantage idna-ssl


In [ ]:
import alpaca_trade_api 

In [ ]:
!ls

sample_data


In [ ]:
!python3 --version

Python 3.6.9


In [ ]:
import alpaca_trade_api as alpacaapi = alpaca.REST('PKEGSKB86SI73AQNJ18U', 'YNUMmC7DoWddJBm6MWAaFmaSJmUZjbdZkhWZOfls', 'https://paper-api.alpaca.markets')

SyntaxError: ignored

In [ ]:
import alpaca_trade_api as alpacaapi = alpaca.REST('PKEGSKB86SI73AQNJ18U', 'YNUMmC7DoWddJBm6MWAaFmaSJmUZjbdZkhWZOfls', 'https://paper-api.alpaca.markets')

SyntaxError: ignored

In [ ]:
$\sqrt{3x-1}+(1+x)^2$

SyntaxError: ignored

In [ ]:
!$\sqrt{3x-1}+(1+x)^2$

/bin/bash: -c: line 0: syntax error near unexpected token `1+x'
/bin/bash: -c: line 0: `$\sqrt{3x-1}+(1+x)^2$'


In [ ]:

import alpaca_trade_api as tradeapi
import time
import datetime
from datetime import timedelta
from pytz import timezone
tz = timezone('EST')

import numpy as np
import pandas as pd

#api = tradeapi.REST('your api key here',
#                    'your api secret code here',
#                    'https://paper-api.alpaca.markets')

api = tradeapi.REST('PKEGSKB86SI73AQNJ18U',
                    'YNUMmC7DoWddJBm6MWAaFmaSJmUZjbdZkhWZOfls',
                    'https://paper-api.alpaca.markets')


import logging
logging.basicConfig(filename='./apca_algo.log', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('{} logging started'.format(datetime.datetime.now().strftime("%x %X")))

def get_data_bars(symbols, rate, slow, fast):

    data = api.get_barset(symbols, rate, limit=20).df

    for x in symbols:
        data.loc[:, (x, 'fast_ema')] = data[x]['close'].rolling(window=fast).mean()
        data.loc[:, (x, 'slow_ema')] = data[x]['close'].rolling(window=slow).mean()
    return data

def get_signal_bars(symbol_list, rate, ema_slow, ema_fast):
    data = get_data_bars(symbol_list, rate, ema_slow, ema_fast)
    signals = {}
    for x in symbol_list:
        if data[x].iloc[-1]['fast_ema'] > data[x].iloc[-1]['slow_ema']: signal = 1
        else: signal = 0
        signals[x] = signal
    return signals

def time_to_open(current_time):
    if current_time.weekday() <= 4:
        d = (current_time + timedelta(days=1)).date()
    else:
        days_to_mon = 0 - current_time.weekday() + 7
        d = (current_time + timedelta(days=days_to_mon)).date()
    next_day = datetime.datetime.combine(d, datetime.time(9, 30, tzinfo=tz))
    seconds = (next_day - current_time).total_seconds()
    return seconds

def run_checker(stocklist):
    print('run_checker started')
    while True:
        # Check if Monday-Friday
        if datetime.datetime.now(tz).weekday() >= 0 and datetime.datetime.now(tz).weekday() <= 4:
            # Checks market is open
            print('Trading day')
            if datetime.datetime.now(tz).time() > datetime.time(9, 30) and datetime.datetime.now(tz).time() <= datetime.time(15, 30):
                signals = get_signal_bars(stocklist, '5Min', 20, 5)
                for signal in signals:
                    if signals[signal] == 1:
                        if signal not in [x.symbol for x in api.list_positions()]:
                            logging.warning('{} {} - {}'.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal]))
                            api.submit_order(signal, 1, 'buy', 'market', 'day')
                            # print(datetime.datetime.now(tz).strftime("%x %X"), 'buying', signals[signal], signal)
                    else:
                        try:
                            api.submit_order(signal, 1, 'sell', 'market', 'day')
                            logging.warning('{} {} - {}'.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal]))
                        except Exception as e:
                            # print('No sell', signal, e)
                            pass

                time.sleep(60)
            else:
                # Get time amount until open, sleep that amount
                print('Market closed ({})'.format(datetime.datetime.now(tz)))
                print('Sleeping', round(time_to_open(datetime.datetime.now(tz))/60/60, 2), 'hours')
                time.sleep(time_to_open(datetime.datetime.now(tz)))
        else:
            # If not trading day, find out how much until open, sleep that amount
            print('Market closed ({})'.format(datetime.datetime.now(tz)))
            print('Sleeping', round(time_to_open(datetime.datetime.now(tz))/60/60, 2), 'hours')
            time.sleep(time_to_open(datetime.datetime.now(tz)))

stocks = ['AA','AAL','AAPL','AIG','AMAT','AMC','AMD',
          'AMGN','AMZN','APA','BA','BABA','BAC','BBY',
          'BIDU','BP','C','CAT','CMG','COP','COST',
          'CSCO','CVX','DAL','DIA','DIS','EBAY',]

print('test:')
print(get_data_bars(['AA'], '5Min', 20, 5).head())

run_checker(stocks)

test:
                              AA                  ...                         
                            open    high     low  ... volume fast_ema slow_ema
2020-07-22 14:20:00-04:00  13.13  13.135  13.110  ...   2337      NaN      NaN
2020-07-22 14:25:00-04:00  13.12  13.120  13.095  ...   3885      NaN      NaN
2020-07-22 14:30:00-04:00  13.11  13.130  13.100  ...   1904      NaN      NaN
2020-07-22 14:35:00-04:00  13.12  13.130  13.110  ...   1180      NaN      NaN
2020-07-22 14:40:00-04:00  13.13  13.130  13.110  ...   1942    13.12      NaN

[5 rows x 7 columns]
run_checker started
Trading day
